# Libraries

In [1]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from utils import *
import cv2

import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.7.1
GPU 사용 가능 여부: True
Tesla P40
1


# Hyperparamters And Seed

In [2]:
batch_size = 16   # Mini-batch size
num_epochs = 35
learning_rate = 0.0001

In [3]:
# seed 고정
random_seed = 21
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# Configuration

In [4]:
class Configuration:
    
    def __init__(self,
                 batch_size = 32,
                 num_epochs = 20,
                 learning_rate = 0.0001,
                 seed = 21,
                 val_every = 1,
                 num_workers = 4,
                 cutmix = False,
                 half = False,
                 train_resize = 224,
                 test_resize = 256,
                 encoder_name = 'senet154',
                 encoder_weights = "imagenet",
                 n_folds = 0, 
                 gkf = False, 
                 skf = False):
        
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.seed = seed
        self.val_every = val_every
        self.num_workers = num_workers
        self.cutmix = cutmix
        self.half = half
        self.train_resize = train_resize
        self.test_resize = test_resize
        self.encoder_name = encoder_name
        self.encoder_weights = encoder_weights
        self.n_folds = n_folds
        self.gkf = gkf
        self.skf = skf
        

In [5]:
config = Configuration()

# WANDB

In [6]:
# !wandb login

In [7]:
# https://app.wandb.ai/authorize

In [8]:
# import wandb

# wandb.init(project="P STAGE 3",
#            config={
#                "batch_size": 32,
#                "learning_rate": 0.0001,
#                "dataset": "COCO Trash",
#            })

# # wandb.log({
# #     'acc':,
# #     'loss':,
# # })

# CutMix

In [9]:
def rand_bbox(size, lam, half=False)->tuple:
    '''
    랜덤한 bounding box의 좌상단,우하단 좌표 반환

    Args:
        size (tuple): batch의 shape
        lam (float): 자를 비율
        half (bool): 절반으로 자름
    '''

    W = size[2]
    H = size[3]

    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)
    
    if half==False:
        bbx1 = np.clip(cx - cut_w // 2, 0, W) 
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)
    else:
        bbx1 = 0
        bby1 = 0
        bbx2 = W//2
        bby2 = H

    return bbx1, bby1, bbx2, bby2

def cutmix(image, mask, alpha, half=False):
    '''
    이미지와 마스크 컷믹스

    Args:
        image (tensor): batch 이미지
        mask (tensor): batch 마스크
        alpha (float): Beta Distribution의 alpha 값
    '''
  
    indices = torch.randperm(image.size(0)) # 배치 크기 입력

    lam = np.clip(np.random.beta(alpha, alpha),0.3,0.4)
    bbx1, bby1, bbx2, bby2 = rand_bbox(image.size(), lam, half)
    new_image = image.clone()
    new_mask = mask.clone()
    new_image[:, :, bby1:bby2, bbx1:bbx2] = image[indices, :, bby1:bby2, bbx1:bbx2]
    new_mask[:, bby1:bby2, bbx1:bbx2] = mask[indices, bby1:bby2, bbx1:bbx2]

    return new_image, new_mask

# Transform

In [10]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90, Rotate, CropNonEmptyMaskIfExists,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

train_transform = Compose([
                            CropNonEmptyMaskIfExists(width = 400, height = 400),
                            CLAHE(clip_limit=2),
                            HorizontalFlip(),
                            GridDistortion(),
                            ShiftScaleRotate(),
                            Resize(512, 512),
                            ToTensorV2()
                            ])

val_transform = Compose([
                        ToTensorV2()
                        ])

test_transform = Compose([
                          ToTensorV2()
                        ])

# Collate Function

In [11]:
def collate_fn(batch):
    return tuple(zip(*batch))

# DataFrame

In [12]:
alldata=pd.read_csv('/opt/ml/code/alldata.csv')
testdata=pd.read_csv('/opt/ml/code/testdata.csv')
display(alldata,testdata)

,image_id,path,width,height,num_objects,bin,UNKNOWN,General trash,Paper,Paper pack,Metal,Glass,Plastic,Styrofoam,Plastic bag,Battery,Clothing
0,0,/opt/ml/input/data/batch_01_vt/0002.jpg,512,512,17,4,0,0,0,0,0,4,4,0,9,0,0
1,1,/opt/ml/input/data/batch_01_vt/0003.jpg,512,512,14,4,0,8,0,0,0,0,0,0,6,0,0
2,2,/opt/ml/input/data/batch_01_vt/0005.jpg,512,512,1,1,0,0,0,0,0,0,1,0,0,0,0
3,3,/opt/ml/input/data/batch_01_vt/0006.jpg,512,512,2,2,0,0,0,0,1,0,1,0,0,0,0
4,4,/opt/ml/input/data/batch_01_vt/0007.jpg,512,512,2,2,0,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3267,3267,/opt/ml/input/data/batch_03/0994.jpg,512,512,7,3,0,0,2,0,0,0,2,2,1,0,0
3268,3268,/opt/ml/input/data/batch_03/0995.jpg,512,512,14,4,0,0,12,0,1,0,0,0,1,0,0
3269,3269,/opt/ml/input/data/batch_03/0996.jpg,512,512,4,2,0,2,1,0,0,0,0,1,0,0,0
3270,3270,/opt/ml/input/data/batch_03/0997.jpg,512,512,8,3,0,0,8,0,0,0,0,0,0,0,0


,image_id,path,width,height
0,0,/opt/ml/input/data/batch_01_vt/0021.jpg,256,256
1,1,/opt/ml/input/data/batch_01_vt/0028.jpg,256,256
2,2,/opt/ml/input/data/batch_01_vt/0031.jpg,256,256
3,3,/opt/ml/input/data/batch_01_vt/0032.jpg,256,256
4,4,/opt/ml/input/data/batch_01_vt/0070.jpg,256,256
...,...,...,...,...
832,832,/opt/ml/input/data/batch_03/0947.jpg,256,256
833,833,/opt/ml/input/data/batch_03/0968.jpg,256,256
834,834,/opt/ml/input/data/batch_03/0969.jpg,256,256
835,835,/opt/ml/input/data/batch_03/0992.jpg,256,256


# Dataset

In [13]:
category_names = ['Backgroud','UNKNOWN','General trash','Paper','Paper pack','Metal',
                    'Glass','Plastic','Styrofoam','Plastic bag','Battery','Clothing']
data_dir = '/opt/ml/input/data/train_all.json'

class TrashDataset(Dataset):
    """DataFrame format"""
    def __init__(self, dataframe, data_dir, mode = 'Train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        self.df = dataframe
        
    def __getitem__(self, index: int):
        
        image_id = self.df.iloc[index]['image_id']
        path = self.df.iloc[index]['path']
        width = self.df.iloc[index]['width']
        height = self.df.iloc[index]['height']      
        images = cv2.imread(path)
        # images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32) # np.float32 -> np.uint8
        # images /= 255.0
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.uint8)
        
        
        if self.mode == 'Train':
            bin = self.df.iloc[index]['bin']
            ann_ids = self.coco.getAnnIds(imgIds=image_id)
            anns = self.coco.loadAnns(ann_ids)

            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            masks = np.zeros((height,width))
            # Background = 0, Unknown = 1, General trash = 2, ... , Cigarette = 11
            for ann in anns:
                pixel_value = ann['category_id']+1
                masks = np.maximum(self.coco.annToMask(ann)*pixel_value, masks)
            # masks = masks.astype(np.float32) # np.float32 -> np.uint8
            masks = masks.astype(np.uint8)

            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return images, masks, bin
        
        if self.mode == 'Test':
            file_name = path.split('/')[-2:]
            file_name = "/".join(file_name)

            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, file_name
    
    
    def __len__(self) -> int:
        return len(self.df)

# DataLoader

## K Fold를 위한 데이터로더

In [14]:
def get_train_valid_dataloader(df, trn_idx, val_idx,fold):

    from torch.utils.data import DataLoader
    
    data_dir = '/opt/ml/input/data/train_all.json'
    
    # 학습, 벨리데이션 데이터프레임 생성
    train_df = df.iloc[trn_idx,:].reset_index(drop=True)
    valid_df = df.iloc[val_idx,:].reset_index(drop=True)
    
    # 학습, 벨리데이션 데이터셋 생성
    print(f'\n###### Fold:{fold} - Loading Dataset ######\n')
    train_ds = TrashDataset(train_df, data_dir=data_dir, transform=train_transform, mode='Train')
    valid_ds = TrashDataset(valid_df, data_dir=data_dir, transform=val_transform, mode='Train')
    print(f'\n###### Fold:{fold} - Loading Dataset - DONE ######\n')
    
    # 학습, 벨리데이션 데이터로더 생성
    train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size,
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=4,
        collate_fn=collate_fn,
    )
    val_dataloader = DataLoader(
        valid_ds, 
        batch_size=batch_size,
        num_workers=4,
        shuffle=False,
        pin_memory=False,
        collate_fn=collate_fn,
    )
    return train_dataloader, val_dataloader


## Test를 위한 데이터로더

In [15]:
def get_test_dataloader(df):
    data_dir = '/opt/ml/input/data/test.json'
    test_ds = TrashDataset(df,data_dir,'Test',test_transform)

    tst_dataloader = torch.utils.data.DataLoader(
                                                test_ds, 
                                                batch_size=batch_size,
                                                num_workers=4,
                                                shuffle=False,
                                                pin_memory=False,
                                                collate_fn=collate_fn,
                                                )
    
    return tst_dataloader

# Model

In [16]:
import segmentation_models_pytorch as smp
from pprint import pprint
dir(smp)

['DeepLabV3',
 'DeepLabV3Plus',
 'FPN',
 'Linknet',
 'PAN',
 'PSPNet',
 'Unet',
 'UnetPlusPlus',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'base',
 'deeplabv3',
 'encoders',
 'fpn',
 'linknet',
 'pan',
 'pspnet',
 'unet',
 'unetplusplus',
 'utils']

In [17]:
def get_model(encoder_name,encoder_weights,in_channels=3,classes=12):
    model = smp.DeepLabV3Plus(
    encoder_name=encoder_name,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights=encoder_weights,     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=in_channels,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=classes,                      # model output channels (number of classes in your dataset)
)
    return model


In [18]:
encoder_name = "se_resnext101_32x4d"
encoder_weights = "imagenet"
model = get_model(encoder_name,encoder_weights)
model.eval();

In [19]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn

preprocess_input = get_preprocessing_fn(encoder_name, pretrained=encoder_weights)

In [20]:
# 구현된 model에 임의의 input을 넣어 output이 잘 나오는지 test

#model = FCN8s(model = model, num_classes=12)
x = torch.randn([1, 3, 512, 512])
print("input shape : ", x.shape)
out = model(x).to(device)
print("output shape : ", out.size())

model = model.to(device)

input shape :  torch.Size([1, 3, 512, 512])
output shape :  torch.Size([1, 12, 512, 512])


# Train And Test

In [21]:
def validation(fold, epoch, model, valid_dataloader, criterion, device):
    print(f'\n- FOLD:{fold} VALIDATION #{epoch} START - TIME:0\n')
    start_time = time.time()
    model.eval()
    hist = np.zeros((12, 12))
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        for step, (images, masks, _) in enumerate(valid_dataloader):
            
            # images = torch.stack(images).to(device)       # (batch, channel, height, width)
            images = torch.stack(images).float().to(device)       # (batch, channel, height, width)
            masks = torch.stack(masks).long().to(device)  # (batch, channel, height, width)

            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            
            outputs = torch.argmax(outputs, dim=1).detach().cpu().numpy()
            
            hist = add_hist(hist, masks.detach().cpu().numpy(), outputs, n_class=12)
            
        acc, acc_cls, mIoU, fwavacc = label_accuracy_score(hist)    
        avrg_loss = total_loss / cnt
        print(f'VALIDATION #{epoch}  Average Loss: {avrg_loss:.4f}, mIoU: {mIoU:.4f}, acc : {acc:.4f}')
    print(f'\n- FOLD:{fold} VALIDATION #{epoch} DONE - TIME:{time.time()-start_time}\n')
    return avrg_loss, mIoU

In [22]:
def train(fold, num_epochs, model, train_dataloader, valid_dataloader, criterion, optimizer, saved_dir, val_every, device, encoder_name):
    print(f'- Fold:{fold} Training Start - TIME:0\n')
    start_time = time.time()
    best_loss = 9999999
    best_mIoU = 0
    
    # GradScaler 선언
    scaler = torch.cuda.amp.GradScaler()
    
    for epoch in range(num_epochs):
        model.train()
        print(f'- Fold:{fold} Epoch:{epoch+1} Training Start - TIME:0\n')
        epoch_start=time.time()
        for step, (images, masks, bin) in enumerate(train_dataloader):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)
            
            # gpu 연산을 위해 device 할당
            # images, masks = images.to(device), masks.to(device) # uint8 -> float32
            images, masks = images.float().to(device), masks.to(device)

#             #####################################
#             # 50% 확률로 CutMix
#             mix_decision = np.random.rand()
#             if mix_decision < 0.5:
#                 # cutmix(data, target, alpha)
#                 images, masks = cutmix(images, masks, 1.)
#             #####################################
                  
    
            # zero_grad
            optimizer.zero_grad()
            
            ######################################## 
            # amp 실행
            with torch.cuda.amp.autocast():
                # inference
                outputs = model(images).to(device)

                # loss 계산 (cross entropy loss)
                loss = criterion(outputs, masks)
                
            scaler.scale(loss).backward()
            scaler.step(optimizer).step()
            scaler.update()
            
            # loss.backward()
            # optimizer.step()
            ######################################## 
            
            
            
            # step 주기에 따른 loss 출력
            if (step + 1) % 25 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                    epoch+1, num_epochs, step+1, len(train_dataloader), loss.item()))
        
        # validation 주기에 따른 loss 출력 및 best model 저장
        if (epoch + 1) % val_every == 0:
            avrg_loss, mIoU = validation(fold, epoch + 1, model, valid_dataloader, criterion, device)
#             if avrg_loss < best_loss:
#                 print('[loss] Best performance at epoch: {}'.format(epoch + 1))
#                 print('Save model in', saved_dir)
#                 print()
#                 best_loss = avrg_loss
#                 save_model(model, saved_dir, file_name = f'fold[{fold}]_loss_best_{encoder_name}(pretrained).pt')
            if mIoU > best_mIoU:
                print('[mIoU] Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                print()
                best_mIoU = mIoU
                save_model(model, saved_dir, file_name = f'fold[{fold}]_mIoU_best_{encoder_name}(pretrained).pt')
        print(f'- Fold:{fold} Epoch:{epoch+1} Training DONE - TIME:{time.time()-epoch_start}\n')
    print(f'\n- Fold:{fold} Training DONE - TIME:{time.time()-start_time}')

## 모델 저장 함수 정의

In [23]:
# 모델 저장 함수 정의
val_every = 1 
    
def save_model(model, saved_dir, file_name='fcn8s_best_model(pretrained).pt'):

    import os

    os.makedirs(saved_dir, exist_ok=True)

    # 모델 자체를 저장
    torch.save(model, saved_dir + '/'+ file_name)

## 단일 모델

In [24]:
def SingleModelTrain(df = pd.read_csv('/opt/ml/code/alldata.csv')):
    
    from pytorch_toolbelt import losses as L
    
    encoder_name = "se_resnext101_32x4d"
    encoder_weights = "imagenet"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    saved_dir = '/opt/ml/code/saved/single'
    
    ####### validation 크기 조절 #######
    train_size = int(len(df)*0.8) # 80% & 20%
    indices = np.random.permutation(len(df))
    trn_idx = indices[:train_size]
    val_idx = indices[train_size:]
    ##################################
    
    model = get_model(encoder_name,encoder_weights)
    model = model.to(device)
    
    # loss
    # criterion = nn.CrossEntropyLoss()
    # criterion = L.FocalLoss()
    criterion = L.SoftCrossEntropyLoss(smooth_factor = 0.1)
    
    # optimizer
    optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate, weight_decay=1e-6)
    
    train_dataloader, valid_dataloader = get_train_valid_dataloader(df, trn_idx, val_idx, 1)
    
    train(fold=1,
          num_epochs=20, 
          model=model, 
          train_dataloader=train_dataloader, 
          valid_dataloader=valid_dataloader, 
          criterion=criterion, 
          optimizer=optimizer, 
          saved_dir=saved_dir, 
          val_every=val_every, 
          device=device, 
          encoder_name=encoder_name)

In [25]:
# SingleModelTrain()

## Group K Fold
- by = bin

In [26]:
def GKF(dataframe=pd.read_csv('/opt/ml/code/alldata.csv'),data_dir='/opt/ml/input/data/train_all.json',n_splits=5):

    from sklearn.model_selection import GroupKFold
    from pytorch_toolbelt import losses as L

    encoder_name = "se_resnext101_32x4d"
    encoder_weights = "imagenet"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    saved_dir = '/opt/ml/code/saved/gkf'

    # bin 기준 나누기
    gkf = GroupKFold(n_splits=n_splits)
    folds = gkf.split(dataframe.values, y=None, groups=dataframe['bin'].values)

    for i, (trn_idx, val_idx) in enumerate(folds):
        # fold별 모델
        model = get_model(encoder_name,encoder_weights)
        model = model.to(device)
        
        # 데이터 로더
        train_dataloader, valid_dataloader = get_train_valid_dataloader(dataframe, trn_idx, val_idx,fold=i+1)

        # loss
        # criterion = nn.CrossEntropyLoss()
        # criterion = L.FocalLoss()
        criterion = L.SoftCrossEntropyLoss(smooth_factor = 0.1)   
        
        # optimizer
        optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate, weight_decay=1e-6)

        # 학습
        train(fold=i+1,
              num_epochs=20, 
              model=model, 
              train_dataloader=train_dataloader, 
              valid_dataloader=valid_dataloader, 
              criterion=criterion, 
              optimizer=optimizer, 
              saved_dir=saved_dir, 
              val_every=val_every, 
              device=device, 
              encoder_name=encoder_name)


In [27]:
# data_dir = '/opt/ml/input/data/train_all.json'
# alldata = pd.read_csv('/opt/ml/code/alldata.csv')

# GKF(alldata,data_dir,5)

# Multi-Label Stratified K Fold

In [28]:
# !pip install pytorch_toolbelt

In [29]:
# !pip install iterative-stratification

In [30]:
def MLSKF(dataframe=pd.read_csv('/opt/ml/code/alldata.csv'),data_dir='/opt/ml/input/data/train_all.json',n_splits=5):

    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    from pytorch_toolbelt import losses as L

    encoder_name = "se_resnext101_32x4d"
    encoder_weights = "imagenet"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    saved_dir = '/opt/ml/code/saved/mlskf_aug'
    multi_label = ['bin','UNKNOWN','General trash','Paper','Paper pack','Metal',
                'Glass','Plastic','Styrofoam','Plastic bag','Battery','Clothing']

    # multi label 기준 나누기
    mlskf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2021)
    folds = mlskf.split(dataframe.values, y=dataframe[multi_label].values)

    for i, (trn_idx, val_idx) in enumerate(folds):
        
        if i!=4: continue
            
        # fold별 모델
        model = get_model(encoder_name,encoder_weights)
        model = model.to(device)
        
        # 데이터 로더
        train_dataloader, valid_dataloader = get_train_valid_dataloader(dataframe, trn_idx, val_idx,fold=i+1)

        # loss
        # criterion = nn.CrossEntropyLoss()
        # criterion = L.FocalLoss()
        criterion = L.SoftCrossEntropyLoss(smooth_factor = 0.1) 
        
        # optimizer
        optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate, weight_decay=1e-6)

        # 학습
        train(fold=i+1,
              num_epochs=20, 
              model=model, 
              train_dataloader=train_dataloader, 
              valid_dataloader=valid_dataloader, 
              criterion=criterion, 
              optimizer=optimizer, 
              saved_dir=saved_dir, 
              val_every=val_every, 
              device=device, 
              encoder_name=encoder_name)


In [ ]:
data_dir = '/opt/ml/input/data/train_all.json'
alldata = pd.read_csv('/opt/ml/code/alldata.csv')

MLSKF(alldata,data_dir,5)


###### Fold:1 - Loading Dataset ######

loading annotations into memory...
Done (t=4.12s)
creating index...
index created!
loading annotations into memory...
Done (t=4.81s)
creating index...
index created!

###### Fold:1 - Loading Dataset - DONE ######

- Fold:1 Training Start - TIME:0

- Fold:1 Epoch:1 Training Start - TIME:0

Epoch [1/20], Step [25/164], Loss: 1.4671
Epoch [1/20], Step [50/164], Loss: 1.1669
Epoch [1/20], Step [75/164], Loss: 1.1847
Epoch [1/20], Step [100/164], Loss: 0.9252
Epoch [1/20], Step [125/164], Loss: 0.9467
Epoch [1/20], Step [150/164], Loss: 0.9224

- FOLD:1 VALIDATION #1 START - TIME:0

VALIDATION #1  Average Loss: 0.8190, mIoU: 0.3931, acc : 0.8959

- FOLD:1 VALIDATION #1 DONE - TIME:113.57429814338684

[mIoU] Best performance at epoch: 1
Save model in /opt/ml/code/saved/mlskf_aug

- Fold:1 Epoch:1 Training DONE - TIME:793.8039793968201

- Fold:1 Epoch:2 Training Start - TIME:0

Epoch [2/20], Step [25/164], Loss: 0.9250
Epoch [2/20], Step [50/164], Loss

## Stratified K Fold
- by = bin

In [ ]:
data_dir = '/opt/ml/input/data/train_all.json'

def SKF(dataframe=pd.read_csv('/opt/ml/code/alldata.csv'),data_dir='/opt/ml/input/data/train_all.json',n_splits=5):

    from sklearn.model_selection import StratifiedKFold
    from pytorch_toolbelt import losses as L

    encoder_name = "se_resnext101_32x4d"
    encoder_weights = "imagenet"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    saved_dir = '/opt/ml/code/saved/skf'

    # bin 기준 나누기
    skf = StratifiedKFold(n_splits=n_splits)
    folds = skf.split(dataframe.values, y=dataframe['bin'].values)

    for i, (trn_idx, val_idx) in enumerate(folds):
        # fold별 모델
        model = get_model(encoder_name,encoder_weights)
        model = model.to(device)
        
        # 데이터 로더
        train_dataloader, valid_dataloader = get_train_valid_dataloader(dataframe, trn_idx, val_idx,fold=i+1)

        # loss
        # criterion = nn.CrossEntropyLoss()
        # criterion = L.FocalLoss()
        criterion = L.SoftCrossEntropyLoss(smooth_factor = 0.1) 

        # optimizer
        optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate, weight_decay=1e-6)

        # 학습
        train(fold=i+1,
              num_epochs=20, 
              model=model, 
              train_dataloader=train_dataloader, 
              valid_dataloader=valid_dataloader, 
              criterion=criterion, 
              optimizer=optimizer, 
              saved_dir=saved_dir, 
              val_every=val_every, 
              device=device, 
              encoder_name=encoder_name)


In [ ]:
# data_dir = '/opt/ml/input/data/train_all.json'
# alldata = pd.read_csv('/opt/ml/code/alldata.csv')

# SKF(alldata,data_dir,5)

# Inference

In [ ]:
def inference(folder_path):

    from glob import glob
    from scipy import stats

    model_list = glob(folder_path + '/*')
    test = pd.read_csv('/opt/ml/code/testdata.csv')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    size = 256

    n_folds = len(model_list)

    soft_voting = [] # Fold,Data,12,256,256
    hard_voting = [] # Fold,Data,256,256
    

    for fold in range(n_folds):

        print(f'\n@@@@@@@@@ FOLD {fold+1} INFERENCE START @@@@@@@@@ - TIME:0\n')
        start_time = time.time()
        
        file_name_list = []
        preds_array = np.empty((0, size*size), dtype=np.long)
        one_soft = [] # Data,12,256,256

        print(f'\n- FOLD {fold+1} DATASET LOAD START -\n')
        test_dataloader = get_test_dataloader(test)
        model = torch.load(model_list[fold])
        model.eval()
        print(f'\n- FOLD {fold+1} DATASET LOAD DONE -\n')
        
        with torch.no_grad():
            for step, (imgs, image_infos) in enumerate(test_dataloader):
                
                if (step+1)%5==0 or step+1==len(test_dataloader):
                    print(f'STEP [{step+1}/{len(test_dataloader)}]')
                
                # inference (512 x 512)
                # outs = model(torch.stack(imgs).to(device)) # batch, 12, 512, 512
                outs = model(torch.stack(imgs).float().to(device)) # batch, 12, 512, 512

                ######### soft voting #########
                soft = outs.detach().cpu().numpy()
                soft = soft.transpose(0,2,3,1)
                
                channel_list = []
                for image in soft:
                    transformed_mask = test_transform(image=image)['image']
                    channel_list.append(transformed_mask)

                # (batch, 12, 256, 256)
                soft = torch.stack(channel_list)
                soft = soft.numpy()
                
                one_soft.append(soft)
                ################################

                ######### hard voting #########
                oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
                
                # resize (256 x 256)
                temp_mask = []
                for img, mask in zip(np.stack(imgs), oms):
                    transformed = A.Compose([A.Resize(size, size)])(image=img, mask=mask)
                    mask = transformed['mask']
                    temp_mask.append(mask)

                oms = np.array(temp_mask)
                
                oms = oms.reshape([oms.shape[0], size*size]).astype(int)
                preds_array = np.vstack((preds_array, oms))

                file_name_list.append([i for i in image_infos])
                
        # soft voting
        one_soft = np.array(one_soft)
        one_soft = np.concatenate(one_soft)
        soft_voting.append(one_soft)
        print(f'soft voting size: {one_soft.shape}')

        # hard voting
        hard_voting.append(preds_array)
        print(f'hard voting size: {preds_array.shape}')
        print(f'\n@@@@@@@@@ FOLD {fold+1} INFERENCE DONE @@@@@@@@@ - TIME:{time.time()-start_time}\n')
        
        # file names
        if fold == 1:
            file_names = [y for x in file_name_list for y in x]

        del model
        torch.cuda.empty_cache()
        
    return file_names, soft_voting, hard_voting

# submission.csv 생성

In [ ]:
# test set에 대한 prediction
folder_path = '/opt/ml/code/saved/mlskf_aug'
encoder_name = "se_resnext101_32x4d"
file_names, soft_voting, hard_voting = inference(folder_path)

# Post Process

In [ ]:
def ensemble_process(soft_voting,hard_voting,size):
    print(f'\n---------- SOFT VOTING START ---------- TIME:0\n')
    start_time = time.time()
    soft_voting = np.concatenate(soft_voting,axis=0,dtype=np.float16) # Fold,Data,12,256,256
    print(soft_voting.shape)
    soft_voting = np.sum(soft_voting, axis=0) # Data,12,256,256
    soft_voting = np.argmax(soft_voting, axis=1) # Data,256,256
    soft_voting = soft_voting.reshape([soft_voting.shape[0], size*size]).astype(int)# Data,256*256
    print(f'\n---------- SOFT VOTING DONE ---------- TIME:{time.time()-start_time}\n')
    print(f'\n---------- HARD VOTING START ---------- TIME:0\n')
    start_time = time.time()
    hard_voting = np.array(hard_voting) # Fold,Data,256*256
    hard_voting = stats.mode(hard_voting)[0] # Data,256*256
    hard_voting = np.squeeze(hard_voting)
    print(f'\n---------- HARD VOTING DONE ---------- TIME:{time.time()-start_time}')
    return soft_voting, hard_voting

In [ ]:
size = 256
soft_voting, hard_voting = ensemble_process(soft_voting, hard_voting, size)

# Soft Voting

In [ ]:
from tqdm import tqdm

submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

for file_name, string in tqdm(zip(file_names, soft_voting)):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)
submission.to_csv(f"/opt/ml/code/submission/{encoder_name}(pretrained)_soft_voting.csv", index=False)

In [ ]:
softdata = pd.read_csv(f"/opt/ml/code/submission/{encoder_name}(pretrained)_soft_voting.csv")
softdata

# Hard Voting

In [ ]:
from tqdm import tqdm

submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
    
for file_name, string in tqdm(zip(file_names, hard_voting)):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

submission.to_csv(f"/opt/ml/code/submission/{encoder_name}(pretrained)_hard_voting.csv", index=False)

In [ ]:
harddata = pd.read_csv(f"/opt/ml/code/submission/{encoder_name}(pretrained)_hard_voting.csv")
harddata